In [1]:
import scipy.sparse as sparse
import scipy
print(scipy.__version__)
dir(sparse)

1.11.0


['SparseEfficiencyWarning',
 'SparseWarning',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_base',
 '_bsr',
 '_compressed',
 '_construct',
 '_coo',
 '_csc',
 '_csparsetools',
 '_csr',
 '_data',
 '_dia',
 '_dok',
 '_extract',
 '_index',
 '_lil',
 '_matrix',
 '_matrix_io',
 '_sparsetools',
 '_sputils',
 '_warnings',
 'base',
 'block_diag',
 'bmat',
 'bsr',
 'bsr_array',
 'bsr_matrix',
 'compressed',
 'construct',
 'coo',
 'coo_array',
 'coo_matrix',
 'csc',
 'csc_array',
 'csc_matrix',
 'csgraph',
 'csr',
 'csr_array',
 'csr_matrix',
 'data',
 'dia',
 'dia_array',
 'dia_matrix',
 'diags',
 'dok',
 'dok_array',
 'dok_matrix',
 'extract',
 'eye',
 'find',
 'hstack',
 'identity',
 'issparse',
 'isspmatrix',
 'isspmatrix_bsr',
 'isspmatrix_coo',
 'isspmatrix_csc',
 'isspmatrix_csr',
 'isspmatrix_dia',
 'isspmatrix_dok',
 'isspmatrix_lil',
 'kron',
 'kronsum',
 'lil',
 'lil_array',
 'lil_matrix',
 

In [2]:
from GmGM.dataset import Dataset
from GmGM.core.preprocessing import center, grammify
from GmGM.core.core import calculate_eigenvectors, calculate_eigenvalues
from GmGM.core.presparse_methods import recompose_sparse_precisions